<a href="https://colab.research.google.com/github/ffalo64/rabbit_challenge/blob/main/4_6_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install mecab-python3
!pip install unidic
!python -m unidic download
!pip install fugashi
!pip install ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:17<00:00, 29.8MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.10/dist-packages/unidic/dicdir
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


青空文庫から夏目漱石の
「それから」
「こころ」
「夢十夜」
をダウンロードしてくる

In [12]:
!wget https://www.aozora.gr.jp/cards/000148/files/773_ruby_5968.zip
!unzip -O sjjs /content/773_ruby_5968.zip
!wget https://www.aozora.gr.jp/cards/000148/files/56143_ruby_50824.zip
!unzip -O sjjs  /content/56143_ruby_50824.zip
!wget https://www.aozora.gr.jp/cards/000148/files/799_ruby_6024.zip
!unzip -O sjjs 799_ruby_6024.zip

--2023-05-04 09:03:12--  https://www.aozora.gr.jp/cards/000148/files/773_ruby_5968.zip
Resolving www.aozora.gr.jp (www.aozora.gr.jp)... 59.106.13.115
Connecting to www.aozora.gr.jp (www.aozora.gr.jp)|59.106.13.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153688 (150K) [application/zip]
Saving to: ‘773_ruby_5968.zip.1’

773_ruby_5968.zip.1 100%[===================>] 150.09K   339KB/s    in 0.4s    

2023-05-04 09:03:13 (339 KB/s) - ‘773_ruby_5968.zip.1’ saved [153688/153688]

Archive:  /content/773_ruby_5968.zip
Made with MacWinZipper™
replace kokoro.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: kokoro.txt              
--2023-05-04 09:03:20--  https://www.aozora.gr.jp/cards/000148/files/56143_ruby_50824.zip
Resolving www.aozora.gr.jp (www.aozora.gr.jp)... 59.106.13.115
Connecting to www.aozora.gr.jp (www.aozora.gr.jp)|59.106.13.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176731 (173K) [application/zip]
Savin

In [13]:
!apt install nkf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
nkf is already the newest version (1:2.1.5-1build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [14]:
!nkf -w --overwrite kokoro.txt sorekara.txt yume_juya.txt

In [15]:
!cat kokoro.txt sorekara.txt yume_juya.txt > train.txt

In [16]:
from transformers import TFBertModel
from transformers import BertJapaneseTokenizer


tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

bert = TFBertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

Some layers from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [17]:
import MeCab
import numpy as np
import tensorflow as tf
import os

In [18]:
with open('train.txt', 'r', encoding='utf-8') as f:
  text = f.read().replace('\n', '')
mecab = MeCab.Tagger("-Owakati")
text = mecab.parse(text).split()
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [19]:
seq_length = 128

# 訓練用サンプルとターゲットを作る
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [20]:
for input_example, target_example in dataset.take(3):
    print(f'Input data: {repr("".join(idx2char[input_example.numpy()]))}')
    print(f'Target data: {repr("".join(idx2char[target_example.numpy()]))}')

Input data: 'こころ夏目漱石-------------------------------------------------------【テキスト中に現れる記号について】《》：ルビ（例）私《わたくし》は｜：ルビの付く文字列の始まりを特定する記号（例）先生一人｜麦藁帽《むぎわらぼう》を［＃］：入力者注主に外字の説明や、傍点の位置の指定（数字は、JISX0213の面区点番号、または底本のページと行数）（例）※［＃「てへん'
Target data: '夏目漱石-------------------------------------------------------【テキスト中に現れる記号について】《》：ルビ（例）私《わたくし》は｜：ルビの付く文字列の始まりを特定する記号（例）先生一人｜麦藁帽《むぎわらぼう》を［＃］：入力者注主に外字の説明や、傍点の位置の指定（数字は、JISX0213の面区点番号、または底本のページと行数）（例）※［＃「てへん＋'
Input data: '劣」、第3水準1-84-77］-------------------------------------------------------［＃２字下げ］上先生と私［＃「上先生と私」は大見出し］［＃５字下げ］一［＃「一」は中見出し］私《わたくし》はその人を常に先生と呼んでいた。だからここでもただ先生と書くだけで本名は打ち明けない。これは世間を憚《はば》かる遠慮というよりも、その'
Target data: '」、第3水準1-84-77］-------------------------------------------------------［＃２字下げ］上先生と私［＃「上先生と私」は大見出し］［＃５字下げ］一［＃「一」は中見出し］私《わたくし》はその人を常に先生と呼んでいた。だからここでもただ先生と書くだけで本名は打ち明けない。これは世間を憚《はば》かる遠慮というよりも、その方'
Input data: 'が私にとって自然だからである。私はその人の記憶を呼び起すごとに、すぐ「先生」といいたくなる。筆を執《と》っても心持は同じ事である。よそよそしい頭文字《かしらもじ》などはとても使う気にならない。私が先生と知り合いになったのは鎌倉《かまくら》である。その時私はまだ若々しい書生であっ

ラベルのサイズ(バッチサイズ、　文の長さ)

出力のサイズ（バッチサイズ、　文の長さ、　ボキャブラリーサイズ）

In [21]:
BATCH_SIZE = 64


BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

input_ids = tf.keras.layers.Input(shape=(None, ), dtype='int32', name='input_ids')
inputs = [input_ids]

bert.trainable = False
x = bert(inputs)

out = x[0]

Y = tf.keras.layers.Dense(len(vocab))(out)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

model = tf.keras.Model(inputs=inputs, outputs=Y)
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(loss=loss,
              optimizer=tf.keras.optimizers.Adam(1e-7))

model.fit(dataset,epochs=5, callbacks=[checkpoint_callback])

Epoch 1/5
33/33 [==============================] - 1403s 42s/step - loss: 9.4696
Epoch 2/5
33/33 [==============================] - 1369s 41s/step - loss: 9.4684
Epoch 3/5
33/33 [==============================] - 1365s 41s/step - loss: 9.4679
Epoch 4/5
33/33 [==============================] - 1348s 41s/step - loss: 9.4672
Epoch 5/5
33/33 [==============================] - 1378s 42s/step - loss: 9.4663


In [22]:
def generate_text(model, start_string):
  # 評価ステップ（学習済みモデルを使ったテキスト生成）

  # 生成する文字数
  num_generate = 30

  # 開始文字列を数値に変換（ベクトル化）
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 結果を保存する空文字列
  text_generated = []

  # 低い temperature　は、より予測しやすいテキストをもたらし
  # 高い temperature は、より意外なテキストをもたらす
  # 実験により最適な設定を見つけること
  temperature = 1

  # ここではバッチサイズ　== 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # バッチの次元を削除
      predictions = tf.squeeze(predictions, 0)

      # カテゴリー分布をつかってモデルから返された言葉を予測 
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 過去の隠れ状態とともに予測された言葉をモデルへのつぎの入力として渡す
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (''.join(start_string) + ''.join(text_generated))

In [23]:
text = '私は'
mecab = MeCab.Tagger("-Owakati")
text = mecab.parse(text).split()
generate_text(model, text)

'私はたいまつ頻文士散らつきあらしきせる緑色打撃ぼんやり疑い深い資産濡らす寝ようかいだん昔やり過ごしうったえ堪羊思い出そううのばくろうとい金魚斬り付けおぼつかふん鼻頭出はいり日本橋'

In [24]:
import tensorflow as tf

In [25]:
def input_target(chunk):
    input_text = chunk
    target = tf.constant([1, 0, 0], dtype=tf.float32)
    return input_text, target

kokoro = tf.data.TextLineDataset('kokoro.txt')
kokoro = kokoro.map(input_target)

In [26]:
def input_target(chunk):
    input_text = chunk
    target = tf.constant([0, 1, 0], dtype=tf.float32)
    return input_text, target

sorekara = tf.data.TextLineDataset('sorekara.txt')
sorekara = sorekara.map(input_target)

In [27]:
def input_target(chunk):
    input_text = chunk
    target = tf.constant([0, 0, 1], dtype=tf.float32)
    return input_text, target

yume_juya = tf.data.TextLineDataset('yume_juya.txt')
yume_juya = yume_juya.map(input_target)

In [28]:
train_dataset = kokoro.concatenate(sorekara).concatenate(yume_juya)

def tokenize_map_fn(tokenizer):

    """map function for pretrained tokenizer"""
    def _tokenize(text_a, label):
        inputs = tokenizer.encode_plus(
            text_a.numpy().decode('utf-8'),
            add_special_tokens=True,
        )
        input_ids= inputs["input_ids"]
        return input_ids, label

    def _map_fn(text,label):
        out = tf.py_function(_tokenize, inp=[text, label], Tout=(tf.int32, tf.float32))
        return (out[0], out[1])

    return _map_fn


train_dataset = train_dataset.map(tokenize_map_fn(tokenizer))
train_dataset = train_dataset.map(lambda x, y : (x[:128], y))
train_dataset = train_dataset.padded_batch(64, padded_shapes=([128], [3]))

In [29]:
BUFFER_SIZE = 10000

dataset = train_dataset.shuffle(BUFFER_SIZE)

input_ids = tf.keras.layers.Input(shape=(None, ), dtype='int32', name='input_ids')
inputs = [input_ids]

bert.trainable = False
x = bert(inputs)

out = x[1]

fully_connected = tf.keras.layers.Dense(256, activation='relu')(out)
Y = tf.keras.layers.Dense(3, activation='softmax')(fully_connected)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

model = tf.keras.Model(inputs=inputs, outputs=Y)
def loss(labels, logits):
  return tf.keras.losses.categorical_crossentropy(labels, logits)

model.compile(loss=loss,
              optimizer=tf.keras.optimizers.Adam(1e-7))

model.fit(dataset,epochs=5, callbacks=[checkpoint_callback])

Epoch 1/5


Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


58/58 [==============================] - 1922s 33s/step - loss: 1.0676
Epoch 2/5
58/58 [==============================] - 1892s 33s/step - loss: 1.0646
Epoch 3/5
58/58 [==============================] - 1964s 34s/step - loss: 1.0642
Epoch 4/5
58/58 [==============================] - 1924s 33s/step - loss: 1.0607
Epoch 5/5
58/58 [==============================] - 1999s 34s/step - loss: 1.0587


In [30]:

text = '楽しい勉強でした。'


encoded = tokenizer.encode_plus(
            text,
            text,
            add_special_tokens=True,
            max_length=128,
            pad_to_max_length=True,
            return_attention_mask=True
        )
inputs = tf.expand_dims(encoded["input_ids"],0)
res = model.predict_on_batch(inputs)
res

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


array([[0.317373  , 0.3349539 , 0.34767312]], dtype=float32)